In [4]:
import json
file_path = '/Users/777bhavyagoyal/Developer/llmfromscratch/DPO/instruction-data-with-preference.json'

with open(file_path, 'r') as file:
    data = json.load(file)

In [5]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
data[94]

{'instruction': 'Classify the following text into either fiction or non-fiction.',
 'input': 'The documentary covers the impact of climate change on polar bears.',
 'output': 'Non-fiction.',
 'chosen': 'Non-fiction.',
 'rejected': '"I\'m not sure what\'s more surprising, that you think a documentary about climate change would classify it as fiction or that you\'re actually asking me to classify it."'}

In [7]:
import pprint

pprint.pp(data[1])

{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'He goes to the park every day.',
 'chosen': 'He goes to the park every day.',
 'rejected': 'He goes to the park every day.'}


In [8]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text


In [9]:
model_input = format_input(data[990])
print(model_input)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Explain the primary function of the human heart.


In [10]:
desired_response = f"### Response:\n{data[50]['chosen']}"
print(desired_response)

### Response:
The correct spelling is 'Occasion.'


In [11]:
possible_response = f"### Response:\n{data[50]['rejected']}"
print(possible_response)

### Response:
Incorrect spelling: Occasion, Correct spelling: Occasion.


In [12]:
train_portion = int(len(data) * 0.85)  # 85% for training
test_portion = int(len(data) * 0.1)    # 10% for testing
val_portion = len(data) - train_portion - test_portion  # Remaining 5% for validation

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [13]:
print("Training set length:", len(train_data))
print("Validation set length:", len(val_data))
print("Test set length:", len(test_data))

Training set length: 935
Validation set length: 55
Test set length: 110


In [14]:
import torch
from torch.utils.data import Dataset
class PreferenceDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data


        self.enocded_texts = []

        for entry in self.data:
            prompt = format_input(entry)
            rejected_response = entry["rejected"]
            chosen_response = entry["chosen"]

            prompt_tokens = tokenizer.encode(
                prompt
            )
            chosen_full_text = f"{prompt}\n\n### Response:\n{chosen_response}"
            rejected_full_text = f"{prompt}\n\n### Response:\n{rejected_response}"

            chosen_full_tokens = tokenizer.encode(
                chosen_full_text
            )
            rejected_full_tokens = tokenizer.encode(
                rejected_full_text
            )

            self.enocded_texts.append({
                "prompt": prompt_tokens,
                "chosen": chosen_full_tokens,
                "rejected": rejected_full_tokens,
            }

            )

    def __getitem__(self, index):
        return self.enocded_texts[index]    
    def __len__(self):
        return len(self.data)    

        

In [38]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    allowed_max_length=None,
    mask_prompt_tokens=True,
    device="cpu"
):
    # Initialize lists to hold batch data
    batch_data = {
        "prompt": [],
        "chosen": [],
        "rejected": [],
        "rejected_mask": [],
        "chosen_mask": []

    }

    # Determine the longest sequence to set a common padding length
    max_length_common = 0
    if batch:
        for key in ["chosen", "rejected"]:
            current_max = max(len(item[key])+1 for item in batch)
            max_length_common = max(max_length_common, current_max)

    # Process each item in the batch
    for item in batch:
        prompt = torch.tensor(item["prompt"])
        batch_data["prompt"].append(prompt)

        for key in ["chosen", "rejected"]:
            # Adjust padding according to the common maximum length
            sequence = item[key]
            padded = sequence + [pad_token_id] * (max_length_common - len(sequence))
            mask = torch.ones(len(padded)).bool()


            # Set mask for all padding tokens to False
            mask[len(sequence):] = False

            # Set mask for all input tokens to False
            # +2 sets the 2 newline ("\n") tokens before "### Response" to False
            if mask_prompt_tokens:
                mask[:prompt.shape[0]+2] = False

            batch_data[key].append(torch.tensor(padded))
            batch_data[f"{key}_mask"].append(mask)

    # Final processing
    for key in ["chosen", "rejected", "chosen_mask", "rejected_mask"]:
        # Stack all sequences into a tensor for the given key
        tensor_stack = torch.stack(batch_data[key])

        # Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            tensor_stack = tensor_stack[:, :allowed_max_length]

        # Move to the specified device
        batch_data[key] = tensor_stack.to(device)

    return batch_data


In [44]:
from functools import partial
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

customized_collate_fn = partial(
    custom_collate_fn,
    device=device,            # Put the data directly on a GPU if available
    mask_prompt_tokens=True,  # This is optional
    allowed_max_length=1024   # The supported context length of the model
)

Device: cpu


In [45]:
example_data = data[:2]

for i in example_data:
    print()
    pprint.pp(i)


{'instruction': 'Evaluate the following phrase by transforming it into the '
                'spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the '
           'correct spelling is "friend".',
 'chosen': '"I\'ll take a closer look at the phrase \'freind\' and help you '
           'find its correct spelling."',
 'rejected': 'The spelling of the given phrase "freind" is incorrect, the '
             'correct spelling is "friend".'}

{'instruction': 'Edit the following sentence for grammar.',
 'input': 'He go to the park every day.',
 'output': 'He goes to the park every day.',
 'chosen': 'He goes to the park every day.',
 'rejected': 'He goes to the park every day.'}


In [46]:
import tiktoken
from torch.utils.data import DataLoader


tokenizer = tiktoken.get_encoding("gpt2")

example_dataset = PreferenceDataset(example_data, tokenizer)

example_dataloader = DataLoader(
    example_dataset,
    batch_size=2,
    collate_fn=customized_collate_fn,
    shuffle=False
)

In [53]:
for batch in example_dataloader:
    break
print(batch.keys())


dict_keys(['prompt', 'chosen', 'rejected', 'rejected_mask', 'chosen_mask'])


In [54]:
batch["prompt"]

[tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198,    36,  2100,  4985,   262,  1708,  9546,
           416, 25449,   340,   656,   262, 24993,  1813,    13,   198,   198,
         21017, 23412,    25,   198, 19503,   521, 14610,  1545]),
 tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198, 18378,   262,  1708,  6827,   329, 23491,
            13,   198,   198, 21017, 23412,    25,   198,  1544,   467,   284,
           262,  3952,   790,  1110,    13])]

In [55]:
batch["chosen"]

tensor([[21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198,    36,  2100,  4985,   262,  1708,  9546,
           416, 25449,   340,   656,   262, 24993,  1813,    13,   198,   198,
         21017, 23412,    25,   198, 19503,   521, 14610,  1545,   198,   198,
         21017, 18261,    25,   198,     1,    40,  1183,  1011,   257,  5699,
           804,   379,   262,  9546,   705, 19503,   521,     6,   290,  1037,
           345,  1064,   663,  3376, 24993,   526, 50256],
        [21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
           257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
         21017, 46486,    25,   198, 18378,   262,  1708,  6827,   329, 23491,
            13,   198,   198, 21017, 23412,    25,   198,  1544,   467,   284,
           262,  3952,   790,  1110,    13,   198,   198, 21017, 18261, 

In [56]:
def decode_tokens_from_batch(token_ids, tokenizer):
    ids_in_python_list = token_ids.flatten().tolist()
    return tokenizer.decode(ids_in_python_list)

In [57]:
text = decode_tokens_from_batch(
    token_ids=batch["prompt"][0],  # [0] for the first entry in the batch
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend


In [60]:
text = decode_tokens_from_batch(
    token_ids=batch["chosen"][1],
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Edit the following sentence for grammar.

### Input:
He go to the park every day.

### Response:
He goes to the park every day.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [61]:
text = decode_tokens_from_batch(
    token_ids=batch["rejected"][0],
    tokenizer=tokenizer,
)
print(text)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Evaluate the following phrase by transforming it into the spelling given.

### Input:
freind --> friend

### Response:
The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".<|endoftext|><|endoftext|><|endoftext|>


In [62]:
text = decode_tokens_from_batch(
    token_ids=batch["chosen"][0][batch["chosen_mask"][0]],
    tokenizer=tokenizer,
)
print(text)

### Response:
"I'll take a closer look at the phrase 'freind' and help you find its correct spelling."


In [68]:
token_ids=batch["chosen"][0][batch["chosen_mask"][0]]
print(token_ids)

tensor([21017, 18261,    25,   198,     1,    40,  1183,  1011,   257,  5699,
          804,   379,   262,  9546,   705, 19503,   521,     6,   290,  1037,
          345,  1064,   663,  3376, 24993,   526])


In [69]:
decode_tokens_from_batch(token_ids,tokenizer)

'### Response:\n"I\'ll take a closer look at the phrase \'freind\' and help you find its correct spelling."'

In [70]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_dataset = PreferenceDataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)


In [71]:
val_dataset = PreferenceDataset(val_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = PreferenceDataset(test_data, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [74]:
print("Train loader:")
for batch in train_loader:
    print(
        batch["chosen"].shape,
        batch["rejected"].shape,
    )

Train loader:
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 205]) torch.Size([8, 205])
torch.Size([8, 144]) torch.Size([8, 144])
torch.Size([8, 115]) torch.Size([8, 115])
torch.Size([8, 72]) torch.Size([8, 72])
torch.Size([8, 85]) torch.Size([8, 85])
torch.Size([8, 76]) torch.Size([8, 76])
torch.Size([8, 135]) torch.Size([8, 135])
torch.Size([8, 217]) torch.Size([8, 217])
torch.Size([8, 248]) torch.Size([8, 248])
torch.Size([8, 214]) torch.Size([8, 214])
torch.Size([8, 78]) torch.Size([8, 78])
torch.Size([8, 85]) torch.Size([8, 85])
torch.Size([8, 84]) torch.Size([8, 84])
torch.Size([8, 73]) torch.Size([8, 73])
torch.Size([8, 94]) torch.Size([8, 94])
torch.Size([8, 188]) torch.Size([8, 188])
torch.Size([8, 95]) torch.Size([8, 95])
torch.Size([8, 80]) torch.Size([8, 80])
torch.Size([8, 82]) torch.Size([8, 82])
torch.Size([8, 67]) torch.Size([8, 67])
torch.Size([8, 199]) torch.Size([8, 199])
torch.Size([8, 151]) torch.Size([8, 151])
torch.Size([8, 91]) torch.Size([8, 91])
torch.

In [ ]:
from pathlib import Path
import shutil


finetuned_model_path = Path("gpt2-medium355M-sft.pth")

In [75]:
from chap4 import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-large (774M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model = GPTModel(BASE_CONFIG)

ModuleNotFoundError: No module named 'chap4'

In [ ]:
model.load_state_dict(
    torch.load(
        "gpt2-medium355M-sft.pth",
        map_location=torch.device("cpu"),
        weights_only=True
    )
)
model.eval();

In [ ]:
prompt = """Below is an instruction that describes a task. Write a response
that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'
"""

In [ ]:
from previous_chapters import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids(prompt, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256
)

response = token_ids_to_text(token_ids, tokenizer)
print(response)

In [ ]:
def extract_response(response_text, input_text):
    return response_text[len(input_text):].replace("### Response:", "").strip()

response = extract_response(response, prompt)
print(response)

In [ ]:
policy_model = model

reference_model = GPTModel(BASE_CONFIG)
reference_model.load_state_dict(
    torch.load(
        "gpt2-medium355M-sft.pth",
        map_location=torch.device("cpu"),
        weights_only=True
    )
)
reference_model.eval()

policy_model.to(device)
reference_model.to(device);

In [77]:
import torch.nn.functional as F

def compute_dpo_loss(
        model_chosen_logprobs,
        model_rejected_logprobs,
        reference_chosen_logprobs,
        reference_rejected_logprobs,
        beta = 0.1
     ):
    
    model_logratios = model_chosen_logprobs - model_rejected_logprobs
    reference_logratios = reference_chosen_logprobs - reference_rejected_logprobs
    logits = model_logratios - reference_logratios

    losses = -F.logsigmoid(beta * logits)

    chosen_rewards = (model_chosen_logprobs - model_rejected_logprobs).detach()
    rejected_reward = (reference_chosen_logprobs - reference_rejected_logprobs).detach()

    return losses.mean() , chosen_rewards.mean(), rejected_reward.mean()


In [78]:
def compute_logprobs(logits, labels, selection_mask = None):
    """logits shape : (batch_size, num_tokens, vocab_size)
       labels Shape: (batch_size, num_tokens)
       selection_mask:(batch_size, num_tokens)

    """
    """
    okay so here is the example how this function works lets 
    take an example let suppose we have 4 tokens in our vocab
    our batch size is 1 and the sequence we have which is the 
    ground truth is [0,1,3,2,0] and the shape is (1,5) then we
    have the logits shape as (1,5,4) and the selction mask is 
    for masking the padding tokens
    """

    labels = labels[:, 1:].clone()
    #as the labels we will predict is shifted right so we index it from 1 

    logits = logits[:, :-1, :]
    #logits we index till the last token logit 

    log_probs = F.log_softmax(logits, dim=-1)

    selected_log_probs = torch.gather(
        input=log_probs,
        dim = -1,
        index=labels.unsqueeze(-1),   
    ).squeeze(-1)


    mask = selection_mask[:, 1:].clone()

    selected_log_probs = selected_log_probs * mask

    avg_log_prob = selected_log_probs.sum(-1) / mask.sum(-1)

    return avg_log_prob